In [183]:
# импортируем необходимые библиотеки
import pandas as pd
import numpy as np
import requests
import sqlite3
import csv
from bs4 import BeautifulSoup

### Создание базы данных

In [184]:
# подключение к базе данных (в случае отсутствия база данных создается)
connection = sqlite3.connect("data/plantarium.db")
# создание объекта подключения
cursor = connection.cursor()
print("База данных создана и успешно подключена к SQLite")

База данных создана и успешно подключена к SQLite


In [99]:
# закрытие соединения с базой данных
#connection.close()

In [185]:
# вывод версии базы данных
sqlite_select_query = "select sqlite_version();"
cursor.execute(sqlite_select_query)
record = cursor.fetchall()
print("Версия базы данных SQLite: ", record)

Версия базы данных SQLite:  [('3.40.1',)]


### Красные книги (парсинг данных)

[Списки охраняемых видов растений и лишайников]("https://www.plantarium.ru/page/redbooks.htmlhttp://www.oopt.aari.ru/rbdata#box%D0%A7")

In [186]:
# адрес сайта
url = "https://www.plantarium.ru/page/redbooks.htmlhttp://www.oopt.aari.ru/rbdata"

In [187]:
# парсинг списка региональных Красных книг
table_books = pd.read_html(url)

In [188]:
# количество распознаных таблиц
len(table_books)

3

In [189]:
# выборка релевантной таблицы
table_book = table_books[1]
table_book.head()

,Unnamed: 0,Регион,Год,Таксоны,Источник,Веб-сайт,Редакторы
0,А,А,А,А,А,А,А
1,1.,Республика Адыгея,2011,169,Постановление от 11 октября 2011 г. № 204 о по...,"oopt.aari.ru, herba.msu.ru",Наталья Гамова
2,2.,Республика Адыгея,2021,200,NaN,docs.cntd.ru,Дмитрий Орешкин
3,3.,Азербайджанская Республика,NaN,138,NaN,–,Николай Дегтярёв
4,4.,Республика Алтай,2007,154,Красная книга Республики Алтай (растения). Гор...,oopt.aari.ru,"Юрий Пирогов, Наталья Гамова"


In [190]:
# подсчет длины наименования региона
table_book["region_name_len"] = table_book["Регион"].map(len)

In [191]:
# исключение алфавитного указателя
table_book = table_book.query("region_name_len > 1")
table_book.head()

,Unnamed: 0,Регион,Год,Таксоны,Источник,Веб-сайт,Редакторы,region_name_len
1,1.,Республика Адыгея,2011,169,Постановление от 11 октября 2011 г. № 204 о по...,"oopt.aari.ru, herba.msu.ru",Наталья Гамова,17
2,2.,Республика Адыгея,2021,200,NaN,docs.cntd.ru,Дмитрий Орешкин,17
3,3.,Азербайджанская Республика,NaN,138,NaN,–,Николай Дегтярёв,26
4,4.,Республика Алтай,2007,154,Красная книга Республики Алтай (растения). Гор...,oopt.aari.ru,"Юрий Пирогов, Наталья Гамова",16
5,5.,Республика Алтай,2017,155,Красная книга Республики Алтай (растения). Гор...,herba.msu.ru,Наталья Гамова,16


In [192]:
# количество уникальных наименований регионов
table_book["Регион"].nunique()

143

In [193]:
# наименование уникальных регионов
table_book.query("region_name_len > 1")["Регион"].unique()[:10]

array(['Республика Адыгея', 'Азербайджанская Республика',
       'Республика Алтай', 'Алтайский край', 'Амурская область',
       'Республика Армения', 'Архангельская область',
       'Астраханская область', 'Балтийский регион (в\xa0работе)',
       'Республика Башкортостан'], dtype=object)

In [195]:
# размерность данных
table_book.shape

(253, 8)

In [196]:
# вывод данных
table_book.head()

,Unnamed: 0,Регион,Год,Таксоны,Источник,Веб-сайт,Редакторы,region_name_len
1,1.,Республика Адыгея,2011,169,Постановление от 11 октября 2011 г. № 204 о по...,"oopt.aari.ru, herba.msu.ru",Наталья Гамова,17
2,2.,Республика Адыгея,2021,200,NaN,docs.cntd.ru,Дмитрий Орешкин,17
3,3.,Азербайджанская Республика,NaN,138,NaN,–,Николай Дегтярёв,26
4,4.,Республика Алтай,2007,154,Красная книга Республики Алтай (растения). Гор...,oopt.aari.ru,"Юрий Пирогов, Наталья Гамова",16
5,5.,Республика Алтай,2017,155,Красная книга Республики Алтай (растения). Гор...,herba.msu.ru,Наталья Гамова,16


### Создание таблицы "Справочник регионы"

* region_id - id наименования региона
* region_name - наименование региона

In [197]:
# создание таблицы regions
cursor.execute("""CREATE TABLE IF NOT EXISTS regions(
                  region_id INTEGER PRIMARY KEY,
                  region_name TEXT UNIQUE);""")
connection.commit()

In [112]:
# удаление таблицы
#cursor.execute("DROP TABLE regions")
#connection.commit()

In [198]:
# наименование уникальных регионов
region_names = table_book.query("region_name_len > 1")["Регион"].unique().tolist()
region_names[:5]

['Республика Адыгея',
 'Азербайджанская Республика',
 'Республика Алтай',
 'Алтайский край',
 'Амурская область']

In [199]:
# преобразование для записи в базу данных
region_names = [(region, ) for region in region_names]
region_names[:5]

[('Республика Адыгея',),
 ('Азербайджанская Республика',),
 ('Республика Алтай',),
 ('Алтайский край',),
 ('Амурская область',)]

In [200]:
# запись в базу данных
cursor.executemany("""INSERT INTO regions (region_name)
                      VALUES(?);""", region_names) 
connection.commit()

In [201]:
# проверка внесения записи
cursor.execute("""SELECT COUNT(*)
                  FROM regions;""")
cursor.fetchone()[0], len(region_names)

(143, 143)

Данные записаны **успешно**

In [202]:
# адрес сайта
url

'https://www.plantarium.ru/page/redbooks.htmlhttp://www.oopt.aari.ru/rbdata'

In [203]:
# парсинг данных с сайта
r = requests.get(url)
r

<Response [200]>

In [204]:
# извлечение разметки страницы
soup = BeautifulSoup(r.text, 'html.parser')

In [205]:
# париснг всех тэгов "a"
list_table = soup.find("table", {"class" : "list-table"}).find_all("a")

In [206]:
# выборка релевантных ссылок
list_table = [row.get("href") for row in list_table if row.get("title") == "Смотреть перечень таксонов"]

In [207]:
# сопоставления данных
table_book.shape, len(list_table)

((253, 8), 253)

In [208]:
# добавление признака в данные
table_book["href"] = list_table

C:\Users\rus_k\AppData\Local\Temp\ipykernel_10912\3992731171.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_book["href"] = list_table


Ссылки собраны **успешно**

### Создание таблицы "Справочник книги"

* book_id - id книги
* book_name - наименование книги (источник)
* book_year - год книги
* book_web - веб сайт книги
* book_href - ссылка на содержание книги
* region_id - id наименования региона

In [209]:
# наименование признаков данных
table_book.columns

Index(['Unnamed: 0', 'Регион', 'Год', 'Таксоны', 'Источник', 'Веб-сайт',
       'Редакторы', 'region_name_len', 'href'],
      dtype='object')

In [210]:
# признаки для записи в базу данных
col_names = ["Регион", "Год", "Источник", "Веб-сайт", "href"]

In [211]:
# вывод данных
table_book[col_names].head()

,Регион,Год,Источник,Веб-сайт,href
1,Республика Адыгея,2011,Постановление от 11 октября 2011 г. № 204 о по...,"oopt.aari.ru, herba.msu.ru",/page/redbook/id/113.html
2,Республика Адыгея,2021,NaN,docs.cntd.ru,/page/redbook/id/322.html
3,Азербайджанская Республика,NaN,NaN,–,/page/redbook/id/58.html
4,Республика Алтай,2007,Красная книга Республики Алтай (растения). Гор...,oopt.aari.ru,/page/redbook/id/65.html
5,Республика Алтай,2017,Красная книга Республики Алтай (растения). Гор...,herba.msu.ru,/page/redbook/id/271.html


In [212]:
# получение id наименования региона из базы данных
def select_region_id(region_name: str) -> int:

    """ функия получения region_id по соответствуюшему region_name"""

    cursor.execute("""SELECT region_id
                    FROM regions
                    WHERE region_name == ?;""", (region_name,))
    
    return cursor.fetchone()[0]

In [213]:
# тест функции select_region_id
select_region_id("Республика Алтай")

3

In [214]:
# проверка на наличие пропусков
table_book[col_names].isna().sum()

Регион       0
Год          3
Источник    14
Веб-сайт     0
href         0
dtype: int64

In [215]:
# создание таблицы books
cursor.execute("""CREATE TABLE IF NOT EXISTS books(
                  book_id INTEGER PRIMARY KEY,
                  book_name TEXT,
                  book_year TEXT,
                  book_web TEXT,
                  book_href TEXT NOT NULL,
                  region_id INTEGER);""")
connection.commit()

In [131]:
# удаление таблицы
#cursor.execute("DROP TABLE books")
#connection.commit()

In [216]:
# запись в базу данных
def insetr_books(book_name:str, book_year:str, book_web:str, book_href:str, region_id:int) -> None:

    """ функция записи данных по книге в базу данных"""

    cursor.execute("""INSERT INTO books (book_name, book_year, book_web, book_href, region_id)
                      VALUES(?, ?, ?, ?, ?);""", (book_name, book_year, book_web, book_href, region_id,)) 
    connection.commit()

In [217]:
# запись в базу данных
for region_name, book_year, book_name, book_web, book_href in table_book[col_names].values:
    region_id = select_region_id(region_name)
    insetr_books(book_name, book_year, book_web, book_href, region_id)

In [218]:
# проверка внесения записи
cursor.execute("""SELECT COUNT(*)
                  FROM books;""")
cursor.fetchone()[0], len(table_book)

(253, 253)

In [219]:
# проверка внесения записи
cursor.execute("""SELECT *
                  FROM books
                  LIMIT 5;""")
books_data = cursor.fetchall()
cursor.execute("PRAGMA table_info('books')")
colnames = [name[1] for name in cursor.fetchall()]
books_df = pd.DataFrame(data=books_data, columns=colnames)
books_df

,book_id,book_name,book_year,book_web,book_href,region_id
0,1,Постановление от 11 октября 2011 г. № 204 о по...,2011,"oopt.aari.ru, herba.msu.ru",/page/redbook/id/113.html,1
1,2,None,2021,docs.cntd.ru,/page/redbook/id/322.html,1
2,3,None,None,–,/page/redbook/id/58.html,2
3,4,Красная книга Республики Алтай (растения). Гор...,2007,oopt.aari.ru,/page/redbook/id/65.html,3
4,5,Красная книга Республики Алтай (растения). Гор...,2017,herba.msu.ru,/page/redbook/id/271.html,3


Данные записаны **успешно**

In [220]:
# ссылка на страницу книги
list_table[0]

'/page/redbook/id/113.html'

In [364]:
href

'/page/redbook/id/56.html'

In [346]:
# адрес сайта
url = "https://www.plantarium.ru"

In [365]:
# парсинг таблиц со сраницы книги
plantarium = pd.read_html(url + '/page/redbook/id/56.html')

In [377]:
url + '/page/redbook/id/56.html'

'https://www.plantarium.ru/page/redbook/id/56.html'

In [366]:
# количество таблиц
len(plantarium)

4

In [367]:
# выбор релевантной таблицы
redbook = plantarium[2]
redbook.head()

,0,1,2,3,4
0,A,A,A,A,NaN
1,1.,NaN,Acantholimon caryophyllaceum Boiss.,Акантолимон гвоздичный,NaN
2,2.,NaN,Acantholimon fedorovii Tamamsch. & Mirzoeva,Акантолимон Фёдорова,NaN
3,3.,NaN,Acantholimon festucaceum (Jaub.et Spach) Boiss.,Акантолимон овсяницевый,NaN
4,4.,NaN,Acanthus dioscoridis L.,Акант Диоскорида,NaN


In [368]:
# подсчет длины наименования растения
redbook["name_len"] = redbook[2].map(len)

In [369]:
# фильтрация алфавитного указателя
redbook = redbook.query("name_len > 1")

In [370]:
# вывод данных
redbook[[2, 3]].head()

,2,3
1,Acantholimon caryophyllaceum Boiss.,Акантолимон гвоздичный
2,Acantholimon fedorovii Tamamsch. & Mirzoeva,Акантолимон Фёдорова
3,Acantholimon festucaceum (Jaub.et Spach) Boiss.,Акантолимон овсяницевый
4,Acanthus dioscoridis L.,Акант Диоскорида
5,Acorus calamus L.,"Аир болотный, Аир обыкновенный, Аир тростниковый"


In [371]:
# вывод данных
redbook[[2, 3]].tail()

,2,3
469,Verbascum megricum (Tzvelev) Hub.-Mor.,Коровяк мегринский
470,Verbascum nudicaule (Wydler) Takht.,"Коровяк голостебельный, Цельзия голостебельная"
471,Vicia pisiformis L.,"Горошек гороховидный, Вика гороховидная"
472,Viola caucasica Kolen. ex Rupr.,Фиалка кавказская
474,Zuvanda meyeri (Boiss.) Askerova,"Зуванда Мейера, Малькольмия Мейера"


In [372]:
# проверка на наличие пропусков
redbook[[2, 3]].isna().sum()

2    0
3    6
dtype: int64

In [297]:
import re

In [355]:
redbook["name_len"] = redbook[3].apply(lambda x: len(re.findall(r"[A-Z]", x)))

C:\Users\rus_k\AppData\Local\Temp\ipykernel_10912\385311571.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  redbook["name_len"] = redbook[3].apply(lambda x: len(re.findall(r"[A-Z]", x)))


In [311]:
# фильтрация алфавитного указателя
redbook = redbook.query("name_len == 0")
redbook.head()

,0,1,2,3,4,name_len,3_len
1,1.,NaN,Alopecurus tiflisiensis (G. Westb.) Grossh.,Лисохвост тифлисский,NaN,0,0
2,2.,NaN,Anacamptis pyramidalis (L.) Rich.,Анакамптис пирамидальный,NaN,0,0
3,3.,NaN,Anaptychia crinalis (Schleich.) Vězda,Анаптихия волосовидная,NaN,0,0
4,4.,NaN,Androsace albana Steven,"Проломник албанский, Проломник азербайджанский...",NaN,0,0
5,5.,NaN,Anemone blanda Schott & Kotschy,"Ветреница нежная, Анемоноидес нежный, Ветренич...",NaN,0,0


### Создание таблицы "Справочник растений RU"

* plant_ru_id - id растения
* plant_ru_name - наименование растения

In [379]:
# создание таблицы plant_ru
cursor.execute("""CREATE TABLE IF NOT EXISTS plant_ru(
                  plant_ru_id INTEGER PRIMARY KEY,
                  plant_ru_name TEXT UNIQUE,
                  plant_lat_id INTEGER);""")
connection.commit()

In [378]:
# удаление таблицы
cursor.execute("DROP TABLE plant_ru")
connection.commit()

In [233]:
# запись в базу данных
def insetr_plant_ru(plant_ru_name:str, plant_lat_id:int) -> None:

    """ функция записи данных по растениям """

    cursor.execute("""INSERT INTO plant_ru (plant_ru_name, plant_lat_id)
                      VALUES(?, ?);""", (plant_ru_name, plant_lat_id,)) 
    connection.commit()

In [314]:
# получение id наименования растения по названию
def select_plant_ru_id(plant_ru_name: str) -> int:

    """ функия получения plant_ru_id по соответствуюшему plant_ru_name """

    cursor.execute("""SELECT plant_ru_id
                    FROM plant_ru
                    WHERE plant_ru_name == ?;""", (plant_ru_name,))
    
    return cursor.fetchone()[0]

### Создание таблицы "Справочник растений LAT"

* plant_lat_id - id растения
* plant_lat_name - наименование растения

In [381]:
# создание таблицы plant_lat
cursor.execute("""CREATE TABLE IF NOT EXISTS plant_lat(
                  plant_lat_id INTEGER PRIMARY KEY,
                  plant_lat_name TEXT UNIQUE);""")
connection.commit()

In [380]:
# удаление таблицы
cursor.execute("DROP TABLE plant_lat")
connection.commit()

In [237]:
# запись в базу данных
def insetr_plant_lat(plant_lat_name:str) -> None:

    """ функция записи данных по растениям """

    cursor.execute("""INSERT INTO plant_lat (plant_lat_name)
                      VALUES(?);""", (plant_lat_name,))
    connection.commit()

In [238]:
# получение id наименования растения по названию
def select_plant_lat_id(plant_lat_name: str) -> int:

    """ функия получения plant_ru_id по соответствуюшему plant_ru_name """

    cursor.execute("""SELECT plant_lat_id
                    FROM plant_lat
                    WHERE plant_lat_name == ?;""", (plant_lat_name,))
    
    return cursor.fetchone()[0]

### Создание таблицы "Содержание книги"

* book_id - id растения
* plant_ru_id - id растения
* plant_lat_id - id растения

In [383]:
# создание таблицы redbooks
cursor.execute("""CREATE TABLE IF NOT EXISTS redbooks(
                  book_id INTEGER,
                  plant_ru_id INTEGER, 
                  plant_lat_id INTEGER);""")
connection.commit()

In [382]:
# удаление таблицы
cursor.execute("DROP TABLE redbooks")
connection.commit()

In [240]:
# запись в базу данных
def insetr_redbooks(book_id:int, plant_ru_id:int, plant_lat_id:int) -> None:

    """ функция записи данных содержания книги по растениям """

    cursor.execute("""INSERT INTO redbooks (book_id, plant_ru_id, plant_lat_id)
                      VALUES(?, ?, ?);""", (book_id, plant_ru_id, plant_lat_id)) 
    connection.commit()

In [241]:
# адрес сайта
url = "https://www.plantarium.ru"

In [242]:
# ссылка на страницу книги
list_table[:3]

['/page/redbook/id/113.html',
 '/page/redbook/id/322.html',
 '/page/redbook/id/58.html']

In [243]:
# получение book_id по href из базы данных
def select_book_id(book_href: str) -> int:

    """ функия получения book_id по соответствуюшему href"""

    cursor.execute("""SELECT book_id
                    FROM books
                    WHERE book_href == ?;""", (book_href,))
    
    return cursor.fetchone()[0]

In [158]:
# тест функции select_book_id
select_book_id("/page/redbook/id/58.html")

3

In [159]:
# адрес сайта
URL = "https://www.plantarium.ru"

In [384]:
# извлечение содержание книги
def get_redbook(href:str) -> np.array:

    """ функция извлечения содержания книги """
    href = URL + href
    # парсинг таблиц со сраницы книги
    plantarium = pd.read_html(href)
    # выбор релевантной таблицы
    redbook = plantarium[2]
    redbook[3] = redbook[3].fillna("нет данных")
    # подсчет длины наименования растения
    redbook["name_len"] = redbook[2].map(len)
    # фильтрация алфавитного указателя
    redbook = redbook.query("name_len > 1")[[2, 3]]
    redbook["name_len"] = redbook[3].apply(lambda x: len(re.findall(r"[A-Z]", x)))
    redbook = redbook.query("name_len == 0")[[2, 3]].values

    return redbook

In [271]:
# href
url + list_table[1]

'https://www.plantarium.ru/page/redbook/id/322.html'

In [319]:
# тест функций
redbook = get_redbook(list_table[0])
redbook[:5]

array([['Alopecurus tiflisiensis (G. Westb.) Grossh.',
        'Лисохвост тифлисский'],
       ['Anacamptis pyramidalis (L.) Rich.', 'Анакамптис пирамидальный'],
       ['Anaptychia crinalis (Schleich.) Vězda',
        'Анаптихия волосовидная'],
       ['Androsace albana Steven',
        'Проломник албанский, Проломник азербайджанский, Проломник восточнокавказский'],
       ['Anemone blanda Schott & Kotschy',
        'Ветреница нежная, Анемоноидес нежный, Ветреничка нежная, Ветреничка приятная']],
      dtype=object)

In [320]:
# тест функций
book_id = select_book_id(list_table[0])
book_id

1

In [321]:
# тест функций
plant_lat_name, plant_ru_names = redbook[0]
plant_lat_name, plant_ru_names

('Alopecurus tiflisiensis (G. Westb.) Grossh.', 'Лисохвост тифлисский')

In [322]:
# тест функций
insetr_plant_lat(plant_lat_name)

In [323]:
# тест функций
plant_lat_id = select_plant_lat_id(plant_lat_name)
plant_lat_id

1

In [327]:
# тест функций
insetr_plant_ru(plant_ru_names, plant_lat_id)

In [328]:
# тест функций
plant_ru_id = select_plant_ru_id(plant_ru_names)
plant_ru_id

1

In [329]:
# тест функций
insetr_redbooks(book_id, plant_ru_id, plant_lat_id)

In [330]:
# тест функций
cursor.execute("""SELECT *
                    FROM redbooks;""")
    
cursor.fetchall()

[(1, 1, 1)]

In [331]:
book_id

1

In [270]:
%%time
# сбор данных по растениям и запись в базу данных
for plant_lat_name, plant_ru_names in redbook:
    plant_ru_names = str(plant_ru_names).strip()
    try:
        insetr_plant_lat(plant_lat_name)
        plant_lat_id = select_plant_lat_id(plant_lat_name)
    except:
        plant_lat_id = select_plant_lat_id(plant_lat_name)
    if len(plant_ru_names.split(',')) > 1:
        plant_ru_names = plant_ru_names.split(',')
        for plant_ru_name in plant_ru_names:
            try:
                insetr_plant_ru(plant_ru_name, plant_lat_id)
                plant_ru_id = select_plant_ru_id(plant_ru_name)
            except:
                plant_ru_id = select_plant_ru_id(plant_ru_name)
            finally:
                insetr_redbooks(book_id, plant_ru_id, plant_lat_id)    
    else:
        try:
            insetr_plant_ru(plant_ru_names, plant_lat_id)
            plant_ru_id = select_plant_ru_id(plant_ru_names)
        except:
            plant_ru_id = select_plant_ru_id(plant_ru_names)
        finally:
            insetr_redbooks(book_id, plant_ru_id, plant_lat_id)

CPU times: total: 719 ms
Wall time: 8.3 s


In [334]:
cursor.execute("""SELECT region_name, plant_ru_name, plant_lat_name, book_name, book_year, book_web
                  FROM regions
                  INNER JOIN books USING(region_id)
                  INNER JOIN redbooks USING(book_id)
                  INNER JOIN plant_ru USING(plant_ru_id)
                  INNER JOIN plant_lat USING(plant_lat_id);""")

plants_data = cursor.fetchall()
colnames = ["Наименование региона", "Название растения RU",
            "Название растения LAT", "Наименование книги (источник)",
           "Год книги", "Веб сайт книги"]
books_df = pd.DataFrame(data=plants_data, columns=colnames)
books_df .head()

,Наименование региона,Название растения RU,Название растения LAT,Наименование книги (источник),Год книги,Веб сайт книги
0,Республика Адыгея,Лисохвост тифлисский,Alopecurus tiflisiensis (G. Westb.) Grossh.,Постановление от 11 октября 2011 г. № 204 о по...,2011,"oopt.aari.ru, herba.msu.ru"
1,Республика Адыгея,Лисохвост тифлисский,Alopecurus tiflisiensis (G. Westb.) Grossh.,Постановление от 11 октября 2011 г. № 204 о по...,2011,"oopt.aari.ru, herba.msu.ru"
2,Республика Адыгея,Анакамптис пирамидальный,Anacamptis pyramidalis (L.) Rich.,Постановление от 11 октября 2011 г. № 204 о по...,2011,"oopt.aari.ru, herba.msu.ru"
3,Республика Адыгея,Анаптихия волосовидная,Anaptychia crinalis (Schleich.) Vězda,Постановление от 11 октября 2011 г. № 204 о по...,2011,"oopt.aari.ru, herba.msu.ru"
4,Республика Адыгея,Проломник албанский,Androsace albana Steven,Постановление от 11 октября 2011 г. № 204 о по...,2011,"oopt.aari.ru, herba.msu.ru"


In [338]:
books_df[books_df["Название растения RU"] == ' T. maximowiczii Korsch.']

,Наименование региона,Название растения RU,Название растения LAT,Наименование книги (источник),Год книги,Веб сайт книги
3689,Амурская область,T. maximowiczii Korsch.,С примечанием s.l.: В пределах Амурской област...,Красная книга Амурской области: Редкие и наход...,2020,–


In [385]:
%%time
# сбор данных по растениям и запись в базу данных
for href in list_table:
    redbook = get_redbook(href)
    book_id = select_book_id(href)
    for plant_lat_name, plant_ru_names in redbook:
        plant_ru_names = str(plant_ru_names)
        try:
            insetr_plant_lat(plant_lat_name)
            plant_lat_id = select_plant_lat_id(plant_lat_name)
        except:
            plant_lat_id = select_plant_lat_id(plant_lat_name)
        if len(plant_ru_names.split(',')) > 1:
            plant_ru_names = plant_ru_names.split(',')
            for plant_ru_name in plant_ru_names:
                try:
                    insetr_plant_ru(plant_ru_name, plant_lat_id)
                    plant_ru_id = select_plant_ru_id(plant_ru_name)
                except:
                    plant_ru_id = select_plant_ru_id(plant_ru_name)
                finally:
                    insetr_redbooks(book_id, plant_ru_id, plant_lat_id)    
        else:
            try:
                insetr_plant_ru(plant_ru_names, plant_lat_id)
                plant_ru_id = select_plant_ru_id(plant_ru_names)
            except:
                plant_ru_id = select_plant_ru_id(plant_ru_names)
            finally:
                insetr_redbooks(book_id, plant_ru_id, plant_lat_id)

TypeError: expected string or bytes-like object, got 'float'

In [182]:
# закрытие соединения с базой данных
connection.close()